In [16]:
from wikidata.client import Client

# Inizializza client
client = Client()

def load_entity(wikidata_id):
    try:
        entity = client.get(wikidata_id, load=True)
        return entity
    except Exception as e:
        print(f"Errore nel caricamento {wikidata_id}: {e}")
        return None

def extract_cultural_features(entity):
    properties = {
        "P495": "country of origin",
        "P2596": "culture",
        "P131": "location",
        "P136": "genre",
        "P31": "instance of",
        "P279": "subclass of",
        "P921": "main subject",
        "P361": "part of"
    }

    results = {}

    for pid, label in properties.items():
        try:
            prop = client.get(pid)
            print(f"\nProcessing property {pid} ({label})...")
            
            if prop in entity:
                # Ottieni direttamente i valori della proprietà
                targets = entity[prop]
                
                # Se è un singolo valore, mettilo in una lista
                if not isinstance(targets, list):
                    targets = [targets]

                labels = []
                
                for target in targets:
                    if hasattr(target, 'label'):
                        label_text = target.label
                        labels.append(label_text)
                    else:
                        print("Target has no label attribute")
                
                results[label] = labels
            else:
                print(f"Property {pid} not found in entity")
                results[label] = []
                
        except Exception as e:
            print(f"Error with property {label}: {e}")
            results[label] = []

    return results

if __name__ == "__main__":
    wikidata_id = "Q7802"  # Bauhaus (non Bauhaus Archive)
    entity = load_entity(wikidata_id)

    if entity:
        features = extract_cultural_features(entity)
        print(f"Feature culturali di {wikidata_id}:\n")
        for prop, values in features.items():
            print(f"- {prop}: {values}")


Processing property P495 (country of origin)...
Property P495 not found in entity

Processing property P2596 (culture)...
Property P2596 not found in entity

Processing property P131 (location)...
Property P131 not found in entity

Processing property P136 (genre)...
Property P136 not found in entity

Processing property P31 (instance of)...

Processing property P279 (subclass of)...

Processing property P921 (main subject)...
Property P921 not found in entity

Processing property P361 (part of)...
Property P361 not found in entity
Feature culturali di Q7802:

- country of origin: []
- culture: []
- location: []
- genre: []
- instance of: [m'type of food or dish']
- subclass of: [m'staple food']
- main subject: []
- part of: []
